This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([202.0924573 , 249.54944341, 210.37520634, 229.79012828,
       241.53770291, 276.21736875, 193.03789201, 407.14718606,
       396.88333541, 340.48947412,  73.74750372, 188.13958381,
       129.67655739, 181.65974631,  71.54661553, 177.34632637,
       233.37177602, 181.75592179, 171.80996061, 146.05982872,
       321.79500281, 381.10780684,  54.94979021, 119.65758775,
       114.36792625, 103.96499392, 228.9145309 , 165.79365236,
       168.46495631, 291.95555798,  45.7531836 ,  37.9332462 ,
        21.6105345 ,  58.99391702,  92.44931641,  24.89260867,
       121.57100715, 183.35068747, 120.21481561,  99.06622089,
       129.12841409,  97.04621619,  60.35669252, 636.33932472,
       209.8692985 ,  67.13987502,  86.43036922,  80.73776665,
       122.31135567, 198.06185912, 237.52127484,   7.667046  ,
        34.7342257 ,  41.60128787, 114.52532705,   0.        ,
         0.        ,   0.        ,  63.15129713,  64.50496061,
         0.        ,  64.66274573,   0.        ,  19.46088327,
        78.02361062,   0.        ,  53.828979  , 130.86141575,
         0.        ,  31.12904856, 177.63303234, 177.44180372,
        74.77205304,  88.18349836, 127.23996661, 212.30252109,
         0.        ,  71.22675703,  41.17090553,  59.05468284,
        40.91463492,  62.16796594,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   1.92094295,   0.        , 169.76938046,
        73.97414683,  12.63281495,  17.33731032,  22.3715509 ,
        16.57505557,  33.43611227,  25.17480383,  40.83543394,
        44.85051669,  39.24127519,  40.26469461,   0.        ,
        39.22644569, 162.30858215, 102.51438554,  14.09444405,
        42.29589311,  29.653255  ,   0.99249062,  11.93148749,
         0.        ,  34.36901768,   0.        ,   0.        ,
         0.        ,  55.37827769,   7.37896699,  23.94141725,
        25.722252  ,  95.52052281,  37.34883385, 208.98907488,
        21.88033825,  47.26926175,  26.16951837,  29.12245041,
        28.2257425 , 150.99987636, 144.19457152,  93.96023403,
        53.54285953,  77.98044279,  59.1838239 ,  21.17208738,
        73.18129625, 125.29220761,   0.        ,   0.        ,
       130.59739217,  93.42580713, 258.66530956, 298.36883132,
        83.91719575, 251.36028695,  39.9617942 ,  42.1689745 ,
       281.10244257, 286.01491902, 175.56939781, 299.69248478,
        62.49404951,  73.13416251,  25.02448734,  13.18080166,
        22.41961241,  33.65498379,  13.89243768,  51.42310236,
         0.        ,   1.40097275,   0.        ,   0.        ,
         0.        ,   0.        ,  72.01080223,  47.92046585,
         0.        ,  90.00434745, 157.15832636,  76.83212346,
        97.08889055,  50.70809969,  40.76042242,  44.32442164,
       169.71976527, 150.56298147, 195.44299332, 106.70124669,
         0.        ,  60.72512971,  81.03949818,  26.50926775,
        26.74964638,  28.58352225,  36.35679348,  23.6410205 ,
        19.2994335 ,  66.76133312,  42.34404359,  71.18482569,
        70.13978156,  94.05845519,   8.63070905,   3.69821609,
        23.21180706,   9.36490231,   0.        ,   0.        ,
        73.75357017, 120.41378893, 191.11304901, 182.07542076,
       186.40027518, 275.65617815, 134.67311657,  97.79037938,
        81.57468903, 224.62287057, 230.11833581, 114.50715188,
       189.82462955, 150.67246806, 192.28465626, 100.86488318,
       152.28019903, 144.0065577 , 190.07525544, 231.48185529,
       118.24855978, 319.11663064, 236.33802852, 250.57926852,
       267.34057988, 294.31041312, 206.36416358, 340.30636296,
       189.64836459, 337.2143761 , 101.88789177, 111.02632779,
       156.18544229, 130.55899831, 195.67275424,  97.70911135,
       212.75663082,  83.54954986,  88.62675986, 177.31783217,
       184.51653892, 280.85429217, 206.61013358,  88.47736135,
       192.71275543, 100.76364613, 213.33322768, 116.27871809,
       185.80703953, 154.70761089, 240.57655817, 329.04283429,
       245.48897712, 338.95548037, 179.99933217, 250.99584507,
       350.05257055, 230.9176424 , 238.98817827, 298.52579257,
       346.90203425, 218.13728473, 164.00223776,  85.49364413,
        84.93486971, 150.04202796, 179.65931277,  79.32480591,
        89.54726266, 165.1097297 ,  54.63840294,  61.57636317,
       129.15559503,  45.29321414, 160.43480672, 158.4134052 ,
        69.20850033,  72.88526591, 180.03516791, 184.33078953,
       199.57809553, 117.79227682, 233.38366567, 324.94508429,
       184.42082504, 132.21158822, 313.927543  , 304.85124615,
       173.74653572, 183.57299247, 287.75549572, 204.55489376,
       956.83195575, 128.63913136,  98.80490655, 107.65517036,
        82.11902013, 123.3670698 , 138.84499588,  78.21525252,
        96.96228032,  77.08989732, 131.79861623, 115.14222467,
        82.82809965, 126.95531231, 130.56858186, 189.42037165,
       102.21705765,  26.61897332,  46.88266895,   9.94733566,
        33.14817606,  62.02428034,  22.02491282,  29.54783309,
        94.26357859, 140.21882844, 124.01781744,  95.20604545,
       188.27143603, 167.32480457, 182.10184863, 211.26795997,
       172.75454393, 236.86191352, 142.93338794, 192.12866913,
       185.54311442, 200.39199842, 220.40870164, 189.99865166,
       108.53738378, 114.75434014, 126.79849001, 188.51471262,
       164.66039086, 145.31046867, 144.11116592, 136.80852088,
       191.31459627,  87.6081324 ,  90.74875619,  69.77529458,
       107.78342612,  29.74770732,  66.96310632,  56.51132963,
        44.18158824,  34.82219739,  46.91723712, 197.61006658,
       283.09312767,  31.31275089,  38.63286983,  95.04900638,
        35.28329862,  93.81143469,  63.58646298, 192.00928341,
       182.41715959, 123.68971028,   0.        ,  82.91347525,
        93.84230286, 124.75938063, 198.10543586,  24.15276207,
       113.48305438,  53.28563087,   0.        ,  61.55780711,
       101.755733  , 160.8491605 ,  63.96485561,  21.35392387,
        33.885364  ,   9.31924847,  61.11953163,  93.06800775,
        28.66375885,   0.        ,  14.2008055 ,  74.81814061,
        17.21211269,   0.        ,  75.6574945 ,  76.84371871,
        82.90154417,  68.97106961,  69.81775523,  55.14080188,
        40.70021975,  60.54108325, 166.79781986,  82.24541371,
        45.49957094,  55.08861603,  43.82195669,  48.24004019,
        46.01034518,   0.        ,   0.        ,   0.        ,
        70.94302761,   0.        ,   0.        ,  87.83887172,
        94.60256203,  28.07241153,  27.25161269,  37.43527934,
        31.26186719,  27.15671258,  59.06262229,  51.90442725,
       125.24500688,   7.0009685 ,  77.68295   ,  13.4709235 ,
        65.77979603,  78.30398606,  25.1156351 ,  38.754046  ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ])
y_pred = array([ 0.00000000e+000,  1.75347022e-005,  8.65474225e-236,
        3.22913951e-143,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  3.21633711e-005,  8.07808194e-249,
        9.26783892e-123,  4.56910113e-028,  9.12064086e-028,
        4.42302845e-192,  4.56759194e-028,  1.89507035e-027,
        3.58067501e-133,  3.02089960e-027,  7.31395945e-133,
        8.25456402e-192,  2.02031130e-010,  3.72703435e-010,
        1.85282313e-304,  2.32522385e-032,  3.05996239e-274,
        3.77609472e-133,  1.77238648e-252,  4.61997034e-032,
        9.86578278e-281,  0.00000000e+000,  0.00000000e+000,
        1.61741939e-008,  7.66926972e-258,  7.27068787e-269,
        5.75754208e-029,  6.18256310e-013,  1.18341055e+002,
        1.00598781e-038,  0.00000000e+000,  1.76499569e-021,
        1.61534783e-132,  1.75037288e-115,  2.08759639e+000,
        1.77066025e-020,  0.00000000e+000,  1.63257790e-183,
        2.15729300e-021,  0.00000000e+000,  2.64374473e+000,
        6.11556778e+001,  2.70519967e-002,  2.07080847e-012,
        0.00000000e+000,  4.21689327e-118,  9.35854751e-099,
        0.00000000e+000,  3.23383289e-158,  0.00000000e+000,
        0.00000000e+000,  9.59903241e-027,  4.20115334e-142,
        0.00000000e+000,  4.21534852e-002, -6.49734257e-014,
        1.28384567e-002,  5.88984428e-005,  3.24968261e-076,
        2.32629736e-083,  3.08802868e-036,  4.30929508e-180,
        9.89400859e-007,  1.57742893e-029,  1.40642069e-047,
        1.77900911e-242,  2.64618467e-131,  4.91228408e-032,
        0.00000000e+000,  0.00000000e+000,  5.85793433e-056,
        5.82975368e-001,  2.46867584e-006,  9.54162851e-001,
        6.88775645e-085,  0.00000000e+000, -3.86900073e-058,
        4.25818944e-013, -1.38194793e-046,  0.00000000e+000,
        6.97177714e-239,  1.96247945e-236,  1.09033740e-004,
        5.89665964e-002,  3.90799562e-084,  1.06255410e-094,
        2.65089394e-001,  2.54340190e-241,  7.29879133e-239,
        2.52805372e-007,  5.99006251e-008,  1.97154806e-012,
        6.91682874e-001,  2.48428775e-006,  2.35391099e-072,
        4.43768392e+000,  0.00000000e+000,  1.23553664e-253,
        8.71218856e-084,  3.32022637e+000,  0.00000000e+000,
        0.00000000e+000,  6.98041852e-005,  4.06989504e-239,
        3.63847095e-072,  4.97738295e-239,  4.69925336e-009,
        6.46174681e-026,  0.00000000e+000,  6.83274234e-096,
        4.30310055e-012,  0.00000000e+000,  5.84546955e-039,
        7.68883791e-013,  1.36657786e-003,  1.16180869e-038,
        6.89642891e-007,  2.53105552e-095,  3.29974370e-012,
        7.65276087e-007,  1.74404155e-072,  4.42589354e-013,
        0.00000000e+000,  2.40953996e-161,  3.93791804e-242,
        8.69064282e-024,  0.00000000e+000,  4.24546355e-012,
        8.57839615e-006,  0.00000000e+000,  1.01368092e-022,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        1.01702160e-022,  1.69322763e-048,  0.00000000e+000,
        0.00000000e+000,  2.00763039e-027,  2.68540700e-105,
        5.58005897e-156,  6.05063252e-083,  1.23023868e-084,
        1.38785794e-001,  1.61439961e-031,  8.88468612e-088,
        8.52163611e-102,  6.37113847e+000,  3.58944412e-008,
        6.27863645e-249,  7.83190318e-088,  0.00000000e+000,
        1.77543543e-240,  0.00000000e+000,  0.00000000e+000,
        5.40193256e-034,  2.02839743e-031,  1.84187820e-013,
        2.21076095e-023,  9.49619697e-102,  1.10463674e+000,
        0.00000000e+000,  2.58836896e-006,  3.14336704e-033,
        4.95572282e-116,  5.74940596e-051,  1.94611415e-006,
        1.43614685e-005,  4.29862572e-002,  3.84477449e-101,
        4.75936101e-006,  1.55868612e-076,  2.70747417e+000,
        0.00000000e+000,  2.03181263e+000,  2.42436425e-101,
        1.97722279e-002,  0.00000000e+000,  0.00000000e+000,
        2.18096121e-017,  0.00000000e+000,  3.73640801e-102,
        3.26642624e-006,  4.10594534e-002,  1.33491259e-007,
        8.87454003e-004,  6.72490932e-009,  3.62060879e-102,
        2.08409087e-200,  7.61643549e-203,  8.36442046e-199,
        1.03687800e-003,  0.00000000e+000,  2.37491584e-006,
        3.22791839e+000,  3.37772214e-001,  4.12002396e+000,
        4.74467210e+000,  0.00000000e+000,  2.46596556e-001,
        2.11922523e-001,  1.15255524e+001,  0.00000000e+000,
        5.55551262e+000,  2.77563440e+000,  3.65494542e-001,
        2.62778254e-001,  4.41212864e-002,  2.35985226e-001,
        1.29893366e-020,  1.53809636e-008,  1.80106770e+001,
        5.58674339e+000,  2.29520597e+000,  1.05161648e-003,
        5.03238955e-001,  5.58081720e-001,  5.80779391e-001,
        7.27582695e-002,  1.63612920e-228,  9.24506597e+000,
        4.70136497e-229,  7.77445803e+000,  1.84401190e-229,
        7.79649230e-230,  2.29194271e+001,  2.87732110e-001,
        4.33427756e-008,  2.13536726e-008,  5.25784929e-001,
        3.21279779e-001,  5.30661876e-001,  2.46044238e-008,
        4.64619191e-002,  3.59419059e-002,  3.82071505e-001,
        2.13790470e-008,  1.93647474e+001,  9.62746865e+000,
        5.22669044e+000,  2.28602310e+000,  3.54533885e+000,
        1.19469208e-010,  5.84063668e-004,  6.69736227e-001,
        5.50267865e-001,  7.50068168e-001,  3.86702492e-008,
        6.14645830e-002,  9.47606487e+000,  7.26842899e+000,
        7.45997558e+000,  9.14736153e+000,  7.08433900e+000,
        5.50563872e+000,  8.01171029e+001,  4.20923986e-230,
        1.27931430e-229,  2.77303252e-001,  4.52935249e-001,
        2.06671831e-001,  2.51113595e-001,  3.13215230e-001,
        1.26320070e-008,  5.73357591e+000,  1.33537102e+001,
        5.35805827e+000,  2.75013789e-047,  8.14689293e-011,
        1.84958101e+000,  1.93120777e+000,  2.03158364e-032,
        4.66922510e-004,  8.59763611e-011,  1.62598635e-004,
        3.54617485e-004,  6.62534298e-001,  4.29921053e-001,
        3.39523674e-002,  8.52551214e+000,  1.85494375e-251,
        5.52069742e+000,  5.79057164e+000,  1.94527666e-236,
        6.70801888e-033,  6.70982502e-010,  3.03867444e-006,
        1.61595020e-070,  5.23458265e-104,  3.06328608e-017,
        1.37959140e-161,  1.53713441e-028,  9.06851584e-020,
        3.66458669e-008,  2.43255016e-025,  1.35693615e-002,
        2.54490482e-001,  6.48694105e-004,  8.55319755e-004,
        1.11754315e-003,  1.51380241e-003,  7.33829095e-004,
        1.94312589e-001,  6.19495089e-015,  1.95358405e-017,
        7.00909933e-011,  1.69761890e-069,  2.77885306e-004,
        3.33501211e-004,  5.66340521e-004,  0.00000000e+000,
        2.48253400e-296,  5.76989405e-305,  4.85138402e-003,
        1.47594322e-158,  4.29466410e-010,  1.69555021e-003,
        9.07922025e-012,  8.29960651e-051,  1.27506600e-003,
        5.50084813e-015,  1.23031011e-002,  1.45555907e-003,
        1.70412993e-083,  1.28395543e-003,  2.21950445e+000,
        3.96457700e+000,  1.50554482e-003,  0.00000000e+000,
        0.00000000e+000,  9.20454814e-005,  1.99137422e+001,
        9.40980826e-005,  4.71723493e+000,  2.15181131e+000,
        0.00000000e+000,  5.98241514e-002,  3.87509666e+000,
        1.32281171e+000,  3.27527275e-004,  6.66364565e-001,
        7.45313741e-026,  5.71711770e-133,  4.39996085e-162,
        1.00320431e-132,  1.22729862e-012,  8.66938227e-007,
        1.87312748e-008,  4.59970137e-036,  1.71958172e-021,
        5.47324846e-015,  6.32571598e-003,  1.13860334e-008,
        8.58253529e-022,  5.37883456e-024,  0.00000000e+000,
        2.60459876e-013,  9.26888146e-018,  3.39376810e-005,
        2.10666454e-053,  1.87738522e+000,  3.47696235e-080,
        1.30424841e-017,  0.00000000e+000,  1.10190048e-001,
        0.00000000e+000,  0.00000000e+000,  3.60359269e-078,
        3.76290045e-028,  3.76471173e-011,  6.72251429e-019,
        0.00000000e+000,  0.00000000e+000,  7.58887726e-002,
        0.00000000e+000,  0.00000000e+000,  3.78487655e-078,
        2.53378165e-003,  0.00000000e+000,  1.07929430e-027,
        1.12937926e-010,  2.03552237e-017,  1.58189371e-065,
        1.11810965e-028,  1.01123964e-005,  1.25012447e-057,
        4.94866468e-006,  3.33584392e-033,  4.71974673e-012,
        6.88711905e-028,  6.37423361e-031,  6.43370494e-011,
        3.26007360e-063,  6.52699403e-013,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  1.13130783e-001,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        3.14896060e-080,  0.00000000e+000,  0.00000000e+000,
        6.29385207e-031,  8.06503557e-018,  4.14477155e-017,
        8.73643218e-008,  5.99916502e-028,  4.37964164e-009,
        3.21338855e-089,  2.01906192e-004,  1.24511624e-020,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        3.63540670e-013,  0.00000000e+000,  0.00000000e+000])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/split_8/test_residuals_histogram.png'
stats = OrderedDict([('R2', 0.0022248053463967965), ('root_mean_squared_error', 148.2758978167523), ('mean_absolute_error', 108.91671025453226), ('rmse_over_stdev', 1.579327396924595)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
